In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [25]:
from transformers import BertTokenizer, BertModel, pipeline

def load_bert_base():
    """
    Load Hugging Face's original BERT-base (uncased) model.
    - 12 layers
    - 12 attention heads
    - Hidden size 768
    """
    model_name = "bert-base-uncased"
    
    # Load tokenizer and model
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    return tokenizer, model


if __name__ == "__main__":
    tokenizer, model = load_bert_base()

    # Example input
    text = "BERT is a powerful language model."
    inputs = tokenizer(text, return_tensors="pt")

    # Forward pass (get hidden states)
    outputs = model(**inputs)

    # The last hidden state representation
    last_hidden_state = outputs.last_hidden_state

    print("Input text:", text)
    print("Tokenized input IDs:", inputs["input_ids"])
    print("Last hidden state shape:", last_hidden_state.shape)


Input text: BERT is a powerful language model.
Tokenized input IDs: tensor([[  101, 14324,  2003,  1037,  3928,  2653,  2944,  1012,   102]])
Last hidden state shape: torch.Size([1, 9, 768])


In [26]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [16]:
from transformers import pipeline

# Load BERT fill-mask pipeline
unmasker = pipeline("fill-mask", model="bert-base-uncased")

# Sentence with a [MASK] token
text = "We are people of [MASK]."

# Get predictions
predictions = unmasker(text)

for pred in predictions:
    print(f"{pred['sequence']} (score: {pred['score']:.4f})")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


we are people of peace. (score: 0.1621)
we are people of god. (score: 0.0769)
we are people of faith. (score: 0.0533)
we are people of honor. (score: 0.0413)
we are people of light. (score: 0.0259)
